### :
    使用Module来自定义层，从而可以被重复调用

In [1]:
import torch
import numpy as np
import sys

### 不含模型参数的自定义层:

In [2]:
class CLayer(torch.nn.Module):
    def __init__(self,**kwargs):
        super(CLayer,self).__init__(**kwargs)
        
    def forward(self,x):
        return x - x.mean()

In [3]:
layer = CLayer()
layer(torch.tensor([1,2,3,4,5],dtype=torch.float)) # 实例化这个层，然后做前向计算

tensor([-2., -1.,  0.,  1.,  2.])

In [5]:
# 也可以用它来构造更复杂的模型
net = torch.nn.Sequential(torch.nn.Linear(8,4),CLayer())
y = net(torch.randn(2,8))
y

tensor([[ 0.0300,  0.5723, -0.7477, -1.0604],
        [ 1.0017, -0.1270,  0.3220,  0.0091]], grad_fn=<SubBackward0>)

### 含模型参数的自定义层
    自定义含模型参数的自定义层。其中的模型参数可以通过训练学出
    
    如果一个 Tensor 是 Parameter，那么它会自动被添加到模型的参数列表里。所以在自定义含模型参数的层时，应该将参数定义成 Parameter
    
    除了直接定义成 Parameter 类外，还可以使用 ParameterList 和 ParameterDict 分别定义参数的列表和字典

In [14]:
class MyListDense(torch.nn.Module):
    def __init__(self):
        super(MyListDense,self).__init__()
        self.params = torch.nn.ParameterList([torch.nn.Parameter(torch.randn(4,4)) for i in range(3)])
        self.params.append(torch.nn.Parameter(torch.randn(4,1)))
        
    def forward(self,x):
        for i in range(len(self.params)):
            x = torch.mm(x,self.params[i])
            
        return x

In [15]:
net = MyListDense()
net

MyListDense(
  (params): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 4x4]
      (1): Parameter containing: [torch.FloatTensor of size 4x4]
      (2): Parameter containing: [torch.FloatTensor of size 4x4]
      (3): Parameter containing: [torch.FloatTensor of size 4x1]
  )
)

In [16]:
# 而 ParameterDict 接收一个 Parameter 实例的字典作为输入然后得到一个参数字典,然后可以按照字典的规则使用了
class MyDictDense(torch.nn.Module):
    def __init__(self):
        super(MyDictDense,self).__init__()
        self.params = torch.nn.ParameterDict({
            'linear1':torch.nn.Parameter(torch.randn(4,4)),
            'linear2':torch.nn.Parameter(torch.randn(4,1))
        })
        
        self.params.update({'linear3':torch.nn.Parameter(torch.randn(4,2))})
    def forward(self,x,i='linear1'):
        out = torch.mm(x,self.params[i])
        return out

In [17]:
net = MyDictDense()
net

MyDictDense(
  (params): ParameterDict(
      (linear1): Parameter containing: [torch.FloatTensor of size 4x4]
      (linear2): Parameter containing: [torch.FloatTensor of size 4x1]
      (linear3): Parameter containing: [torch.FloatTensor of size 4x2]
  )
)

In [19]:
# 使用自定义层构造模型
net = torch.nn.Sequential(
    MyDictDense(),
    MyListDense()
)
net

Sequential(
  (0): MyDictDense(
    (params): ParameterDict(
        (linear1): Parameter containing: [torch.FloatTensor of size 4x4]
        (linear2): Parameter containing: [torch.FloatTensor of size 4x1]
        (linear3): Parameter containing: [torch.FloatTensor of size 4x2]
    )
  )
  (1): MyListDense(
    (params): ParameterList(
        (0): Parameter containing: [torch.FloatTensor of size 4x4]
        (1): Parameter containing: [torch.FloatTensor of size 4x4]
        (2): Parameter containing: [torch.FloatTensor of size 4x4]
        (3): Parameter containing: [torch.FloatTensor of size 4x1]
    )
  )
)